In [11]:
# combine/load listings data

import pandas as pd

cities = ["barcelona", "prague", "mallorca", "lisbon", 
          "malaga", "athens", "vienna", "crete", 
          "florence", "london", "edinburgh", 
          "rome", "paris", "copenhagen", "amsterdam"]

print("loading cities data...")

try: 
    cities_data = pd.read_csv("data/listings.csv.gz")
    print("data loaded")
except:
    print("data not found. preparing data")
    cities_data = pd.DataFrame()

    for city in cities:
        print(f"loading {city} data...")
        city_data = pd.read_csv(f"data/{city}-listings.csv.gz")
        city_data['city'] = city
        cities_data = pd.concat([cities_data, city_data], axis=0)
    
    print("data prepped and loaded")

cities_data.to_csv("data/listings.csv.gz", index=False)

loading cities data...
data not found. preparing data
loading barcelona data...
loading prague data...
loading mallorca data...
loading lisbon data...
loading malaga data...
loading athens data...
loading vienna data...
loading crete data...
loading florence data...
loading london data...
loading edinburgh data...
loading rome data...
loading paris data...
loading copenhagen data...
loading amsterdam data...
data prepped and loaded


In [12]:
import polars as pl
df = pl.read_csv('data/listings.csv.gz')

df = df.filter(pl.col('city')=='barcelona')
df = df.filter((pl.col('property_type')=='Entire rental unit')&(pl.col('has_availability')=='t'))

In [13]:
host_count = df['host_id'].value_counts().sort('count')
breaks = [1, 2, 5, 10, 100]
labels = ['1', '2', '2+', '5+', '10+', '100+']
host_count = host_count.with_columns(
    pl.col('count').cut(breaks=breaks, labels=labels, left_closed=False).alias('binned_counts')
)
host_count

host_id,count,binned_counts
i64,u32,cat
9100900,1,"""1"""
18078971,1,"""1"""
355379355,1,"""1"""
50349408,1,"""1"""
12070015,1,"""1"""
…,…,…
265193861,144,"""100+"""
4459553,191,"""100+"""
32037490,196,"""100+"""


In [14]:
df = df.join(host_count, on='host_id', how='left')

graph_data = df['binned_counts'].value_counts().sort('binned_counts')
total_sum = graph_data['count'].sum()
graph_data = graph_data.with_columns(((pl.col('count')/total_sum)*100).round().cast(pl.Int32).alias('percentage'))
graph_data

binned_counts,count,percentage
cat,u32,i32
"""1""",1929,19
"""2""",640,6
"""2+""",1001,10
"""5+""",1002,10
"""10+""",3837,38
"""100+""",1619,16


In [26]:
import plotly.express as px

palette = ["#537c78","#7ba591","#cc222b","#f15b4c","#faa41b","#ffd45b"]

# text annotation is written manually so as to manually modify the x position
text_annotation = ['19%', '6%', '10%', '10%', '38%', '16%']
text_annotation_xpos = [17, 5, 8, 8, 35, 15]
text_annotation_ypos = [5, 4, 3, 2, 1, 0]
annotations_text = [
    dict(x=x, y=y, text=text_annotation, showarrow=False, font=dict(color='white', weight='bold', size=20))
    for x, y, text_annotation in zip(text_annotation_xpos, text_annotation_ypos, text_annotation)
]

fig = px.bar(graph_data, x='percentage', y='binned_counts', orientation='h', color='binned_counts',
             color_discrete_sequence=palette,
             category_orders={'binned_counts': ['1', '2', '2+', '5+', '10+', '100+']}
        )

fig.update_layout(
    height = 700,
    width = 1100,
    template = 'plotly_white',
    annotations = annotations_text,
    xaxis_title = 'Percentage of listings',
    yaxis_title = 'Number of listings owned by the same host',
    title = dict(text="Prevalence of multi-property in Barcelona's Airbnb listings<br><sup>% of Airbnb listings in Barcelona owned by multiproperty hosts</sup>", font=dict(size=30)),
    font = dict(family="Sans-serif"),
    legend = dict(
        orientation='h',
        x=0.5,
        y=-0.13,
        xanchor='center',
        yanchor='bottom',
        title="Number of properties per host"
    )
)

fig.update_yaxes(anchor='free', shift=-10, tickfont=dict(size=18, weight='normal'))
fig.show()

In [29]:

def airbinb_per_host(file, ptype, neighborhood):
    df = pl.read_csv(file)

    if neighborhood:
        df = df.filter((pl.col('property_type')==ptype)&(pl.col('neighborhood_group_cleansed')==neighborhood)&
                       (pl.col('has_availability')=='t'))
    else:
        df = df.filter((pl.col('property_type')==ptype)&(pl.col('has_availability')=='t'))
    
    host_count = df['host_id'].value_counts().sort('count')
    breaks = [1, 2, 5, 10, 100]
    labels = ['1', '2', '2+', '5+', '10+', '100+']
    host_count = host_count.with_columns(
        pl.col("count").cut(breaks=breaks, labels=labels, left_closed=False).alias('binned_counts')
    )
    df = df.join(host_count, on='host_id', how='left')

    graph_data = df['binned_counts'].value_counts().sort('binned_counts')
    total_sum = graph_data['count'].sum()
    graph_data = graph_data.with_columns(((pl.col('count')/total_sum)*100).alias('percentage'))

    return graph_data



In [30]:
import os
import glob

df_combined = pl.DataFrame({
    "binned_counts": pl.Series(dtype=pl.Categorical),
    "count": pl.Series(dtype=pl.UInt32),
    "percentage": pl.Series(dtype=pl.Float64),
    "city": pl.Series(dtype=pl.Categorical)
})

city_files = glob.glob("data/*.csv.gz")

for file in city_files:
    file_name = os.path.basename(file)
    city = file_name.split("-")[0]
    print(f"processing {city} data...")

    data = airbinb_per_host(file, 'Entire rental unit', None)

    data = data.with_columns(pl.lit(city.capitalize()).alias('city'))
    df_combined = pl.concat([df_combined, data], how='vertical')

print(f"finished processing of {len(city_files)} files")

processing listings.csv.gz data...


sys:1: CategoricalRemappingWarning:

Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance



SchemaError: type String is incompatible with expected type Categorical(Some(local), Physical)